In [1]:
#### specify path of Julia Code for 2D Models ####
### By default the working directory ./2D_Models ###
path = string( pwd(), "/2D_Models/")

"/home/jakob/Dokumente/Repositories/Modeling_Framework/2D_Models/"

In [2]:
##### include 2D Modelling Framework ####
include(string(path, "./2D_model_library.jl"));


########## OPERATIONAL ##########
##### how to chose the actual velocity? #####

#op_model = "Constant"
#op_model = "Collision_Free_Speed"
#op_model = "RVO"
#op_model = "SocialForce"
#op_model = "AV_Model"
op_model = "AV_IN_Model"
#op_model = "IN_Costfunction"

#op_model = "Intrusion_Force"

include_op_model(op_model, path)

########## TACTICAL ##########
#### how to chose the preferred velocity? ###

tact_model = "Constant"
#tact_model = "IN_Model"

include_tact_model(tact_model, path)


########## STRATEGIC ##########
### how to chose the desired velocity ###

strat_model = "Constant"
#strat_model = "Towards_Goal"

include_strat_model(strat_model, path)


########## OPTIMIZATION SCHEME ##########
### If optimization of a cost-function is included, how to solve? ###

#optimization_scheme = "Regular_Sampling"
optimization_scheme = "Random_Sampling"

include_optimization_scheme(optimization_scheme, path)


##### which update scheme? #####

update = "Parallel_Update"
#update = "Step_Update"

include_update(update, path)


##### which order? i.e. Acceleration or Velocity based? ####
#order = "first"
order = "second"

include_order(order, path);

In [9]:
#### parameters of the agents ####
p = [1.0, 0.0, 2.0, 1.0, 0.3, 0.0, 0.0, 0.1, 5.0, 8.0, 0.0, 0.0, 0.5, 0.0]
# ModelParameter: v_max, v_des, T, T2, l, step_time, τ_A, τ_R, α, β, ζ_h, ζ_v, r, ϕ
p_desc = "v_max, T, T2, l, step_time, τ_A, τ_R, α, β, ζ_h, ζ_v, r, ϕ"

### of the system ###
ρ = 2.55#3.74

system_size = (5.0, 5.0) #important for periodic boundaries
L = 5
N = Int(round(ρ*L*L))

### of the simulation ###
sim_p = [0.0, 5.0, 0.05, 0.01, 3.0] #t_relax, t_max, dt_save, dt, r
sim_p_desc = "t_relax, t_max, dt_save, dt, r"

### initialize geometry ###
geometrie = create_geometry_single_obstacle((37.5,31.5), 0.3)

### initialize crowd ###
menge = create_crowd(N, geometrie)
Init_Hom_Parameters!(p, menge)
Init_Hom_Vels!(menge, 0.0)

CrowdOneSide!(menge, 0.4, 0.0, L)

In [ ]:
positions, headings = Simulate!(menge, geometrie, sim_p[1], sim_p[2], sim_p[3], sim_p[4], sim_p[5], system_size);

In [ ]:
using GR
GR.inline("mov")

j = 1

dt_save = sim_p[3]
geometry_x, geometry_y = rewrite_geometry(geometrie)
boundaries_x, boundaries_y = boundaries_rectangle_positions(system_size)

while j*sim_p[3] < sim_p[2]-sim_p[1]
    
        GR.clearws()

        GR.setwindow(-1, 11, -1, 11)
        GR.setviewport(0.0, 1, 0.0, 1)    

        plot_boundaries(boundaries_x, boundaries_y, 0.2, 999)        

        plot_agents(positions[j, :], headings[j, :], 2.3, 100, 0.7, 999, 0.1)
    
        model = op_model
        timer = string("time ", round(j*dt_save, digits = 1), "s")
        GR.text(0.7, 0.95, timer)
        GR.text(0.7, 0.9, model)
        
        GR.updatews()
    
        j = j+1
end

In [ ]:
GR.show()

In [23]:
Path = "/home/jakob/Dokumente/Sim_Data/"
dir = "Waiting_AV"
header = Header(p, p_desc, sim_p, sim_p_desc, op_model, tact_model, order, update, N, system_size);

Save_Data!(Path, dir, header, positions)

In [ ]:
name = "Waiting"
path = "/home/jakob/Dokumente/Renne_Software/UMANS_3/examples/agents/"

#Set_Goal_from_E_des!(menge, (99.0, 0.0))
Change_Coordinate_System_Agents!(menge, -1 .*(system_size./2))
Write_Agents_XML(menge, name, path)

In [ ]:
path = "/home/jakob/Dokumente/Renne_Software/WithWaiting/examples/"
policy = "Moussaid"
agents = "Waiting_"

name = string(agents, policy)


Write_Scenario_XML(path, name, sim_p, system_size, policy, agents)